In [1]:
import ibis
from ibis import _
import leafmap.maplibregl as leafmap
con = ibis.duckdb.connect(extensions=["spatial"])
parquet = "/home/jovyan/huggingface/datasets/boettiger-lab/pad-us-3/pad-stats.parquet"
pad_df = con.read_parquet(parquet)

#pad_df.group_by(_.manager_type).agg(n=_.count()).execute()
#pad_df.filter(_.designation_type == "Conservation Easement").group_by(_.state).agg(n=_.count()).execute()
x = con.read_geo("/home/jovyan/huggingface/datasets/boettiger-lab/pad-us-3/pad-stats.pmtiles")

In [6]:
pad_df.mutate(geometry = _.geometry.convert("ESRI:54009","epsg:4326")).to_parquet("/home/jovyan/huggingface/datasets/boettiger-lab/pad-us-3/pad-stats-4326.parquet")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [2]:
import geopandas as gpd 
gdf = gpd.read_parquet(parquet)

In [3]:
gdf.crs

<Projected CRS: ESRI:54009>
Name: World_Mollweide
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Mollweide
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [24]:
easement = pad_df.filter(_.designation_type == "Conservation Easement", _.state.isin(["WY", "MO", "ID"]))

easement.group_by(_.manager_name).agg(n = _.count()).execute()

,manager_name,n
0,Natural Resources Conservation Service,691
1,Other or Unknown Local Government,3
2,Other or Unknown State Land,10
3,U.S. Fish and Wildlife Service,204
4,Bureau of Land Management,52
5,State Department of Conservation,4
6,State Fish and Wildlife,19
7,National Park Service,217
8,Non-Governmental Organization,348
9,Forest Service,43


In [17]:
import leafmap.maplibregl as leafmap
m = leafmap.Map(center=(-110, 43.50), zoom=6, style="positron")
m.add_layer_control()

m.add_basemap("Esri.NatGeoWorldMap")


hi="https://data.source.coop/vizzuality/hfp-100/hfp_2021_100m_v1-2_cog.tif"
m.add_cog_layer(hi, palette="purples", name="Human Impact", transparent_bg=True, opacity = 0.7, fit_bounds=False)

deforest_carbon = "https://data.source.coop/vizzuality/lg-land-carbon-data/deforest_carbon_100m_cog.tif"
m.add_cog_layer(deforest_carbon, palette="blues", name="deforested carbon", transparent_bg=True, opacity = 0.8, fit_bounds=False)

m.add_cog_layer("https://data.source.coop/vizzuality/lg-land-carbon-data/natcrop_expansion_100m_cog.tif",  palette="oranges", name="cropland expansion", transparent_bg=True, opacity = 0.8, fit_bounds=False)

m.add_cog_layer("https://data.source.coop/vizzuality/lg-land-carbon-data/natcrop_bii_100m_cog.tif", palette="reds", name="biodiversity intactness loss", transparent_bg=True, opacity = 0.8, fit_bounds=False)
    
   
m.add_cog_layer("https://data.source.coop/vizzuality/lg-land-carbon-data/natcrop_fii_100m_cog.tif",  palette="greens", name="forest integrity loss", transparent_bg=True, opacity = 0.8, fit_bounds=False)




In [18]:
pad_pmtiles = "https://huggingface.co/datasets/boettiger-lab/pad-us-3/resolve/main/pad-stats.pmtiles"
style = {
    "version": 8,
    "sources": {
        "pad": {
            "type": "vector",
            "url": "pmtiles://" + pad_pmtiles,
            "filter": ['==', 'designation_type', 'Conservation Easement'],
            "attribution": "US PAD v3"}},
    "layers": [{
            "id": "Easements",
            "source": "pad",
            "source-layer": "pad-stats",
            "type": "fill",
            "paint": {
                "fill-color": "#FFA07A",
                "fill-opacity": 0.8
            }
        }]}

In [19]:
m.add_pmtiles(pad_pmtiles, style=style, visible=True, tooltip=False,   fit_bounds=False)
#m.to_html("yellowstone_impacts.html")
m


Map(height='600px', map_options={'bearing': 0, 'center': (-110, 43.5), 'pitch': 0, 'style': 'https://tiles.ope…